Import from Drive

In [1]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Install requirements

In [2]:
!pip install sentence-transformers==2.2.2
!pip install numpy==1.25.0
!pip install transformers==4.30.2
!pip install pytorch_lightning==2.0.6
!pip install tensorboard==2.13.0
!pip install black==23.7.0
!pip install spacy==3.7.2
!pip install torch==2.0.1
!pip install labml==0.4.167
!pip install matplotlib==3.7.2
!pip install Deprecated==1.2.14
!pip install sympy==1.12
!pip install nltk==3.8.1
!pip install dataclasses==0.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=98c83e9e64d4feff18308629a73a0fd8b635d959e5af84b0f5699066378b0563
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    

ERROR: Ignored the following versions that require a different python version: 0.7 Requires-Python >=3.6, <3.7; 0.8 Requires-Python >=3.6, <3.7
ERROR: Could not find a version that satisfies the requirement dataclasses==0.8 (from versions: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6)
ERROR: No matching distribution found for dataclasses==0.8


Append File Path

In [1]:
# Path to the directory containing your Python file
# path_to_file = '/content/drive/Othercomputers/My Mac/cl_model/'
path_to_file = '/content/drive/MyDrive/cl_model_new/'

# !ls '/content/drive/MyDrive/cl_model/'

# Add the path to sys.path to access the module
import sys
sys.path.append(path_to_file)

# Now import the module
from category_router.category_router import CategoryRouter


Main Training Code

In [ ]:
import unittest
from typing import List, Dict, Any

import torch
from torch import nn
import time

from category_router.category_router import CategoryRouter
from cl_data.src.constants import Constants
from cl_pretrainer.batch_builder import BatchBuilder
from cl_pretrainer.cl_pre_trainer import ClPreTrainer
from cl_pretrainer.lr_scheduler import NoamOpt
from cl_pretrainer.pre_trainer_checkpoint_manager import ClPreTrainerCheckPointManager
from cl_pretrainer.pre_trainer_utils import PreTrainerUtils
from data_loader.data_loader import DataLoader
from vocabulary_builder.category_vocabulary_builder import CategoryVocabBuilder
from vocabulary_builder.output_vocabulary_builder import OutputVocabBuilder
from evaluation_metric.bleu import calculate_corpus_bleu_score, get_n_gram_weights
from response_parser.response_parser import ResponseParser

CURRENT_BATCH_OUTPUT_LOSS = "current_batch_output_loss"
CURRENT_BATCH_OUTPUT_ACCURACY = "current_batch_output_accuracy"


def cl_pre_trainer_train(
        model: nn.Module,
        category_vocab_builder: CategoryVocabBuilder,
        output_vocab_builder: OutputVocabBuilder,
        scheduler: Any,
        batches: Dict[str, List[List[List[dict]]]],
        masks: Dict[str, List[torch.Tensor]],
        n_epochs: int,
        category_criterion: any,
        output_criterion_map: dict[int, any],
        patience=5,
        start_epoch=0,
        is_training=True,
        verbose_log=False,
        only_language_training=0,
        device: torch.device = torch.device('cpu')
):
    model.train(is_training)
    if not is_training:
        n_epochs = 1

    num_iters = 0
    best_accuracy = 0
    best_loss = float('inf')
    epochs_without_improvement = 0
    execute_epoch = 40
    for epoch in range(start_epoch, start_epoch + n_epochs):
        total_accuracy = 0
        total_loss = 0
        for i, (src_batch, padding_mask, tgt_batch, future_mask, task_types) in enumerate(
                zip(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY],
                    masks[BatchBuilder.PADDING_MASK_KEY],
                    batches[BatchBuilder.DECODER_IO_PARSER_OUTPUT_KEY],
                    masks[BatchBuilder.FUTURE_MASK_KEY],
                    masks[BatchBuilder.TASK_TYPE_KEY],
                    )
        ):
            # Start time
            start_time = time.time()

            # Move to CPU or Cuda
            padding_mask = padding_mask.to(device)
            future_mask = future_mask.to(device)

            # ~~~~~~~~~~~~~~~~~~~~~~~~~ Compute category probability ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            tgt_category_probability = torch.tensor(category_vocab_builder.batch_encoder(tgt_batch)).to(device)
            # Removing the <BOS> category map
            tgt_category_probability = tgt_category_probability[:, 1:]

            e_one = model.category_map_decoder.forward(
                batch_io_parser_output=src_batch,
                task_types=task_types,
                future_mask=future_mask,
                # src_padding_mask=padding_mask,
            )

            category_probability, category_logits = model.category_map_classification_head.forward(e_one)
            category_probability = category_probability[:, :-1]
            category_logits = category_logits[:, :-1, :]

            # Compute the average cross-entropy loss over all next-token predictions at each index i given [1, ..., i]
            # for the entire batch. Note that the original paper uses label smoothing (I was too lazy).
            batch_category_loss = category_criterion(
                category_logits.contiguous().permute(0, 2, 1),
                tgt_category_probability.contiguous().long(),
            )
            total_loss += batch_category_loss

            # Rough estimate of per-token accuracy in the current training batch
            batch_category_accuracy = (torch.sum(
                category_logits.argmax(dim=-1) == tgt_category_probability)) / torch.numel(tgt_category_probability)

            total_accuracy += batch_category_accuracy

            # ~~~~~~~~~~~~~~~~~~~~~~~~~~ Compute output token probability ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            tgt_output_probability = output_vocab_builder.batch_encoder(tgt_batch, is_only_probability=False)

            e_two = model.output_token_decoder.forward(
                batch_io_parser_output=src_batch,
                task_types=task_types,
                future_mask=future_mask,
                # src_padding_mask=padding_mask,
            )

            # As predicting proper category is the responsibility of the left side.
            # That's why we are only concerning ourselves with the output token prediction.
            # Provided we are always using the correct output classification head.
            # That's why using the src_batch instead of the predicted batch category map.
            batch_route_ids = category_vocab_builder.batch_encoder_output_token_classification_head_vocab_items(
                batch_io_parser_output=src_batch,
            )
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~  is_hub=True ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # SO WE ARE ALSO USING output_vocab_builder.batch_decode_for_training
            output_logits_map = model.category_router.forward(
                e_two=e_two,
                batch_route_ids=batch_route_ids,
                is_hub=True,
            )

            combined_output_losses = []
            # Calculate output loss & accuracy for each classification head
            for index, output_logits_item in output_logits_map.items():
                if epoch >= only_language_training or index == 1:
                    current_tgt_output_probability = PreTrainerUtils.create_tgt_tensor_for_output_classification_head(
                        output_classification_head_index=index,
                        tgt_batch_probability=tgt_output_probability,
                    ).to(device)
                    # Removing the <BOS> tgt output probability
                    current_tgt_output_probability = current_tgt_output_probability[:, 1:]

                    current_output_logits = output_logits_item[CategoryRouter.OUTPUT_LOGITS]
                    current_output_logits.to(device)

                    # Removing the last garbage token from output logits
                    current_output_logits = current_output_logits[:, :-1, :]

                    current_batch_output_loss = output_criterion_map[index](
                        current_output_logits.contiguous().permute(0, 2, 1),
                        current_tgt_output_probability.contiguous().long(),
                    )
                    combined_output_losses.append(current_batch_output_loss)

                    current_batch_output_accuracy = (torch.sum(
                        current_output_logits.argmax(dim=-1) == current_tgt_output_probability)) / torch.numel(
                        current_tgt_output_probability)

                    output_logits_item[CURRENT_BATCH_OUTPUT_LOSS] = current_batch_output_loss
                    output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY] = current_batch_output_accuracy
                    total_accuracy += current_batch_output_accuracy
                    total_loss += current_batch_output_loss
                    output_logits_map[index] = output_logits_item
                else:
                    output_logits_item[CURRENT_BATCH_OUTPUT_LOSS] = 100
                    output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY] = 0
                    output_logits_map[index] = output_logits_item


            print_model_training_status(
                [sequence_list[1:] for sequence_list in tgt_batch],
                batch_category_accuracy,
                batch_category_loss,
                batches,
                category_probability,
                category_vocab_builder,
                epoch,
                is_training,
                num_iters,
                output_logits_map,
                output_vocab_builder,
                verbose_log,
            )

            # Update parameters
            if is_training:
                batch_category_loss.backward()
                total_loss = sum(combined_output_losses)
                total_loss.backward()
                scheduler.step()
                scheduler.optimizer.zero_grad()
                if num_iters % len(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY]) == 0:
                    print(f"Current learning rate is {scheduler.get_current_rate()} and running rate is {scheduler.get_rate()}")
                    end_time = time.time()
                    total_time = end_time - start_time
                    print(f"Batch training time: {total_time:.2f} seconds")

            num_iters += 1
            # batch

        # Saving the best model ...
        best_accuracy = save_best_model(best_accuracy, epoch, model, scheduler, total_accuracy)

        # Applying early stopping using the total loss ...
        if total_loss < best_loss:
            best_loss = total_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping after {epoch + 1} epochs with no improvement.")
            execute_epoch = epoch + 1
            break
        # epoch
    # function
    print(f"Best accuracy found: {best_accuracy}")
    return batch_category_loss, batch_category_accuracy, output_logits_map, execute_epoch


def save_best_model(best_accuracy, epoch, model, scheduler, total_accuracy):
    if total_accuracy > best_accuracy and epoch > 2:
        best_accuracy = total_accuracy
        # Saving the best model
        ClPreTrainerCheckPointManager.save_checkpoint_map(
            path=TestClPreTrainerTraining.BEST_PATH,
            epoch=epoch + 1,
            model=model,
            optimizer=scheduler.optimizer,
        )
        print(f"Saved best model at epoch: {epoch + 1} with best accuracy: {best_accuracy}")
    return best_accuracy


def print_model_training_status(
        target_batch,
        batch_category_accuracy,
        batch_category_loss,
        batches,
        category_probability,
        category_vocab_builder,
        e,
        is_training,
        num_iters,
        output_logits_map,
        output_vocab_builder,
        verbose_log):
    if num_iters % len(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY]) == 0 or not is_training:
        print(
            f"epoch: {e}, num_iters: {num_iters}, "
            f"batch_category_loss: {batch_category_loss}, batch_category_accuracy: {batch_category_accuracy}"
        )
        for index, output_logits_item in output_logits_map.items():
            output_loss = output_logits_item[CURRENT_BATCH_OUTPUT_LOSS]
            print(f"output loss for index: {index} is {output_loss}")
        for index, output_logits_item in output_logits_map.items():
            output_accuracy = output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY]
            print(f"output accuracy for index: {index} is {output_accuracy}")

        if verbose_log:
            predicted_category_map = category_vocab_builder.batch_decode(category_probability.tolist())
            print(f"Predicted category probability values:"
                  f" {predicted_category_map}")

            predicted_tokens_map = {}
            for index, output_logits_item in output_logits_map.items():
                current_head_output_probability = output_logits_item[CategoryRouter.OUTPUT_PROBABILITY]
                current_head_output_probability = current_head_output_probability[:, :-1]
                current_head_predicted_output_token = output_vocab_builder.batch_decode_for_training(
                    index,
                    current_head_output_probability.tolist(),
                )
                print(f"Predicted token values for index: {index} is \n"
                      f"{current_head_predicted_output_token}")

                if not is_training:
                    # Get the output token classification vocab item from index
                    current_output_token_classification_head_vocab_item = \
                        output_vocab_builder.index_to_output_vocabularies[index][
                            OutputVocabBuilder.OUTPUT_TOKEN_CLASSIFICATION_HEAD_VOCAB_ITEM]

                    # Add item to predicted tokens map using the output token classification head vocab item as key
                    predicted_tokens_map[current_output_token_classification_head_vocab_item] = {
                        OutputVocabBuilder.PREDICTED_TOKEN_KEY: current_head_predicted_output_token,
                        OutputVocabBuilder.INDEX: index,
                    }
            if not is_training:
                predicted_io_parser_output = PreTrainerUtils.recreate_io_parser_output_hub(predicted_category_map,
                                                                                           predicted_tokens_map,
                                                                                           start_from=1)
                parsed_response_list = ResponseParser.parse_corpus_io_parser_output(predicted_io_parser_output)
                print(f"Response parser output is: {parsed_response_list} ")

                target_batch_extracted_token = PreTrainerUtils.extract_tokens(target_batch)
                predicted_batch_extracted_token = PreTrainerUtils.extract_tokens(predicted_io_parser_output)
                bleu_score = calculate_corpus_bleu_score(
                    target_batch_extracted_token,
                    predicted_batch_extracted_token,
                    bleu_weights=get_n_gram_weights(2),
                )
                print(f"BLEU Score is: {bleu_score}")
        print("\n")

Traing Test 1

In [ ]:
import unittest

class TestClPreTrainerTraining(unittest.TestCase):
    PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_only_last_v1.pth"
    BEST_PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_only_best_v1.pth"
    accepted_loss_threshold = 0.80
    accepted_accuracy_threshold = 0.90

    def test_cl_pre_trainer_train_and_save(self):
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f'\nSelected Device: {device}')

        # Hyperparameters
        n_epochs = 100
        batch_size = 1 if device == torch.device("cpu") else 1
        num_heads = 8
        hidden_dim = 768
        ff_dim = 2048
        num_layers = 6
        dropout_p = 0.1
        max_decoding_length = 24
        task_generator_indexes = [2, 3]
        generator_range = 2 if device == torch.device("cpu") else 98
        number_of_batch = generator_range * len(task_generator_indexes)
        seed = 42
        add_bos_and_eos = True
        warmup = 250 # (number of batch * epoch) // 2
        max_rate= 0.00008
        lr_factor = 0.05 #0.025
        training_batch_size = 40

        # Initializing the data loader
        data_loader = DataLoader()

        data_loader_result = data_loader.create_data_loader_output(
            batch_size=batch_size,
            number_of_batch=number_of_batch,
            add_bos_and_eos=add_bos_and_eos,
            max_sequence_length=max_decoding_length,
            task_generator_indexes=task_generator_indexes,
            generator_indexes=[i for i in range(generator_range)],
            identifier=0,
            shuffle=True,
            seed=seed,
        )
        batch_size = training_batch_size

        print(f"len of data_loader_result: {len(data_loader_result)}")
        corpus_io_parser_output = [item[Constants.IO_PARSER_OUTPUT] for item in data_loader_result]

        # Initialize category vocabulary builder instance
        category_vocab_builder = CategoryVocabBuilder(corpus_io_parser_output)
        category_vocab_size = len(category_vocab_builder.category_vocab_item_to_index.keys())

        # print(f"Output token classification head count:"
        #       f" {len(category_vocab_builder.index_to_output_token_classification_head_vocab_item.keys())}\n"
        #       f"Output token classification head category type:"
        #       f" {category_vocab_builder.index_to_output_token_classification_head_vocab_item}")

        # Initialize output vocabulary builder instance
        output_vocab_builder = OutputVocabBuilder(
            corpus_of_io_parser_output=corpus_io_parser_output,
            index_to_output_token_classification_head_vocab_item=
            category_vocab_builder.index_to_output_token_classification_head_vocab_item
        )
        output_vocabularies = output_vocab_builder.index_to_output_vocabularies
        # print(f"Output vocabularies count: {len(output_vocabularies.keys())}\n")

        # Creating the batch and masks
        batches, masks = BatchBuilder.construct_batches_for_cl_pre_trainer_with_data_loader(
            data_loader_result,
            batch_size=batch_size,
            max_decoder_sequence_length=max_decoding_length,
            is_generative_training=True,
            add_bos_and_eos=add_bos_and_eos
        )
        print(f"\nNumber of batch available is: {len(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY])}\n")
        # Initializing the CL pre trainer
        cl_pre_trainer = ClPreTrainer(
            batch_size=batch_size,
            hidden_dim=hidden_dim,
            ff_dim=ff_dim,
            num_heads=num_heads,
            num_layers=num_layers,
            max_decoding_length=max_decoding_length,
            dropout_p=dropout_p,
            category_vocab_size=category_vocab_size,
            index_to_output_vocabularies=output_vocabularies,
        )
        # Moved to CPU or GPU
        cl_pre_trainer.to(device)
        cl_pre_trainer.eval()
        # Initialize learning rate scheduler, optimizer and loss (note: the original paper uses label smoothing)
        optimizer = torch.optim.AdamW(
            cl_pre_trainer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01,
        )
        scheduler = NoamOpt(
            cl_pre_trainer.hidden_dim,
            factor=lr_factor,
            warmup=warmup,
            optimizer=optimizer,
            max_rate=max_rate,
        )

        category_criterion = PreTrainerUtils.get_category_criterion(
            category_index_to_count=category_vocab_builder.index_to_count,
        )
        output_criterion_map = PreTrainerUtils.get_output_criterion_map(
            index_to_output_vocabularies=output_vocab_builder.index_to_output_vocabularies,
        )

        # Start training and verify ~zero loss and >90% accuracy on the last batch
        latest_batch_loss, latest_batch_accuracy, output_logits_map, execute_epoch = cl_pre_trainer_train(
            model=cl_pre_trainer,
            category_vocab_builder=category_vocab_builder,
            output_vocab_builder=output_vocab_builder,
            scheduler=scheduler,
            batches=batches,
            masks=masks,
            n_epochs=n_epochs,
            is_training=True,
            verbose_log=False,
            category_criterion=category_criterion,
            output_criterion_map=output_criterion_map,
            patience=80,
            device=device
        )

        # Saving the model...
        ClPreTrainerCheckPointManager.save_checkpoint_map(
            path=TestClPreTrainerTraining.PATH,
            epoch=n_epochs,
            model=cl_pre_trainer,
            optimizer=optimizer,
        )

        print(f"batch loss {latest_batch_loss.item()}")
        print(f"batch accuracy {latest_batch_accuracy}")
        self.assertEqual(latest_batch_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
        self.assertEqual(latest_batch_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)
        for index, output_logits_item in output_logits_map.items():
            output_loss = output_logits_item[CURRENT_BATCH_OUTPUT_LOSS]
            output_accuracy = output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY]
            self.assertEqual(output_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
            self.assertEqual(output_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)

test_suite = unittest.TestLoader().loadTestsFromTestCase(TestClPreTrainerTraining)
unittest.TextTestRunner(verbosity=2).run(test_suite)

test_cl_pre_trainer_train_and_save (__main__.TestClPreTrainerTraining) ... 


Selected Device: cuda
len of data_loader_result: 196

Number of batch available is: 32

epoch: 0, num_iters: 0, batch_category_loss: 5.559696674346924, batch_category_accuracy: 0.0054347822442650795
output loss for index: 0 is 1.1791969537734985
output loss for index: 1 is 5.670948028564453
output loss for index: 2 is 7.85086727142334
output loss for index: 3 is 5.409491539001465
output loss for index: 4 is 5.372064590454102
output loss for index: 5 is 4.029409408569336
output loss for index: 6 is 1.3596681356430054
output loss for index: 7 is 2.9426937103271484
output loss for index: 8 is 2.6747965812683105
output loss for index: 9 is 0.9060867428779602
output loss for index: 10 is 0.9868746995925903
output accuracy for index: 0 is 0.44999998807907104
output accuracy for index: 1 is 0.004347825888544321
output accuracy for index: 2 is 0.19456520676612854
output accuracy for index: 3 is 0.0021739129442721605
output accuracy for index: 4 is 0.0
output accuracy for index: 5 is 0.0010869

KeyboardInterrupt: 

**Training the saved model...**

In [ ]:
import unittest

class TestClPreTrainerTraining(unittest.TestCase):
    SAVED_MODEL = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_best_v4.pth"
    PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_last_v5.pth"
    BEST_PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_best_v5.pth"
    accepted_loss_threshold = 0.80
    accepted_accuracy_threshold = 0.90

    def test_cl_pre_trainer_train_and_save(self):
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f'\nSelected Device: {device}')

        # Hyperparameters
        n_epochs = 40
        batch_size = 2 if device == torch.device("cpu") else 4
        num_heads = 8
        hidden_dim = 768
        ff_dim = 2048
        num_layers = 6
        dropout_p = 0.1
        max_decoding_length = 24
        task_generator_indexes = [0,1,2]
        generator_range = 2 if device == torch.device("cpu") else 20
        number_of_batch = generator_range * len(task_generator_indexes)
        seed = 42
        add_bos_and_eos = True
        warmup = 8000 # (number of batch * epoch) // 2
        max_rate= 0.00005
        lr_factor = 0.05 #0.025
        training_batch_size = 4

        # Initializing the data loader
        data_loader = DataLoader()

        data_loader_result = data_loader.create_data_loader_output(
            batch_size=batch_size,
            number_of_batch=number_of_batch,
            add_bos_and_eos=add_bos_and_eos,
            max_sequence_length=max_decoding_length,
            task_generator_indexes=task_generator_indexes,
            generator_indexes=[i for i in range(generator_range)],
            identifier=0,
            shuffle=True,
            seed=seed,
        )
        batch_size = training_batch_size

        print(f"len of data_loader_result: {len(data_loader_result)}")
        corpus_io_parser_output = [item[Constants.IO_PARSER_OUTPUT] for item in data_loader_result]

        # Initialize category vocabulary builder instance
        category_vocab_builder = CategoryVocabBuilder(corpus_io_parser_output)
        category_vocab_size = len(category_vocab_builder.category_vocab_item_to_index.keys())

        # print(f"Output token classification head count:"
        #       f" {len(category_vocab_builder.index_to_output_token_classification_head_vocab_item.keys())}\n"
        #       f"Output token classification head category type:"
        #       f" {category_vocab_builder.index_to_output_token_classification_head_vocab_item}")

        # Initialize output vocabulary builder instance
        output_vocab_builder = OutputVocabBuilder(
            corpus_of_io_parser_output=corpus_io_parser_output,
            index_to_output_token_classification_head_vocab_item=
            category_vocab_builder.index_to_output_token_classification_head_vocab_item
        )
        output_vocabularies = output_vocab_builder.index_to_output_vocabularies
        # print(f"Output vocabularies count: {len(output_vocabularies.keys())}\n")

        # Creating the batch and masks
        batches, masks = BatchBuilder.construct_batches_for_cl_pre_trainer_with_data_loader(
            data_loader_result,
            batch_size=batch_size,
            max_decoder_sequence_length=max_decoding_length,
            is_generative_training=True,
            add_bos_and_eos=add_bos_and_eos
        )
        print(f"\nNumber of batch available is: {len(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY])}\n")
        # Initializing the CL pre trainer
        cl_pre_trainer = ClPreTrainer(
            batch_size=batch_size,
            hidden_dim=hidden_dim,
            ff_dim=ff_dim,
            num_heads=num_heads,
            num_layers=num_layers,
            max_decoding_length=max_decoding_length,
            dropout_p=dropout_p,
            category_vocab_size=category_vocab_size,
            index_to_output_vocabularies=output_vocabularies,
        )
        # Moved to CPU or GPU
        cl_pre_trainer.to(device)
        cl_pre_trainer.eval()

        # Initialize learning rate scheduler, optimizer and loss (note: the original paper uses label smoothing)
        optimizer = torch.optim.AdamW(
            cl_pre_trainer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01,
        )

        # Load the model...
        checkpoint_map = ClPreTrainerCheckPointManager.load_checkpoint_map(
            TestClPreTrainerTraining.SAVED_MODEL
        )
        # Load CL-Pre-Trainer states
        cl_pre_trainer.load_saved_model_from_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.CL_PRE_TRAINER_STATE,
            ),
        )
        # Load Optimizer states
        optimizer.load_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OPTIM_STATE,
            ),
        )
        start_epoch = ClPreTrainerCheckPointManager.get_checkpoint_item(
            checkpoint_map,
            ClPreTrainerCheckPointManager.EPOCH,
        )

        # Load Category map Output token classification heads state
        cl_pre_trainer.category_router.load_all_output_classification_head(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OUTPUT_TOKEN_CLASSIFICATION_HEADS_STATE,
            ),
        )
        print("Model loaded correctly...")

        scheduler = NoamOpt(
            cl_pre_trainer.hidden_dim,
            factor=lr_factor,
            warmup=warmup,
            optimizer=optimizer,
            max_rate=max_rate,
        )

        category_criterion = PreTrainerUtils.get_category_criterion(
            category_index_to_count=category_vocab_builder.index_to_count,
        )
        output_criterion_map = PreTrainerUtils.get_output_criterion_map(
            index_to_output_vocabularies=output_vocab_builder.index_to_output_vocabularies,
        )

        # Start training and verify ~zero loss and >90% accuracy on the last batch
        latest_batch_loss, latest_batch_accuracy, output_logits_map, execute_epoch = cl_pre_trainer_train(
            model=cl_pre_trainer,
            category_vocab_builder=category_vocab_builder,
            output_vocab_builder=output_vocab_builder,
            scheduler=scheduler,
            batches=batches,
            masks=masks,
            n_epochs=n_epochs,
            is_training=True,
            verbose_log=False,
            category_criterion=category_criterion,
            output_criterion_map=output_criterion_map,
            patience=80,
            device=device
        )

        # Saving the model...
        ClPreTrainerCheckPointManager.save_checkpoint_map(
            path=TestClPreTrainerTraining.PATH,
            epoch=n_epochs,
            model=cl_pre_trainer,
            optimizer=optimizer,
        )

        print(f"batch loss {latest_batch_loss.item()}")
        print(f"batch accuracy {latest_batch_accuracy}")
        self.assertEqual(latest_batch_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
        self.assertEqual(latest_batch_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)
        for index, output_logits_item in output_logits_map.items():
            output_loss = output_logits_item[CURRENT_BATCH_OUTPUT_LOSS]
            output_accuracy = output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY]
            self.assertEqual(output_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
            self.assertEqual(output_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)

test_suite = unittest.TestLoader().loadTestsFromTestCase(TestClPreTrainerTraining)
unittest.TextTestRunner(verbosity=2).run(test_suite)

test_cl_pre_trainer_train_and_save (__main__.TestClPreTrainerTraining) ... 


Selected Device: cuda
len of data_loader_result: 240

Number of batch available is: 499

Model loaded correctly...
epoch: 0, num_iters: 0, batch_category_loss: 0.5256407856941223, batch_category_accuracy: 1.0
output loss for index: 0 is 0.27546143531799316
output loss for index: 1 is 4.038623809814453
output loss for index: 2 is 1.689697265625
output loss for index: 3 is 15.835234642028809
output loss for index: 4 is 28.485240936279297
output loss for index: 5 is 6.616557598114014
output loss for index: 6 is 0.10633242875337601
output loss for index: 7 is 0.5280718207359314
output accuracy for index: 0 is 1.0
output accuracy for index: 1 is 0.97826087474823
output accuracy for index: 2 is 0.8913043737411499
output accuracy for index: 3 is 0.77173912525177
output accuracy for index: 4 is 0.95652174949646
output accuracy for index: 5 is 1.0
output accuracy for index: 6 is 1.0
output accuracy for index: 7 is 1.0


Current learning rate is 2.5214735326871204e-09 and running rate is 2.5214

KeyboardInterrupt: 

Training test 2

In [ ]:
import unittest

class TestClPreTrainerTraining(unittest.TestCase):
    PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_last.pth"
    BEST_PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_best.pth"
    accepted_loss_threshold = 0.40
    accepted_accuracy_threshold = 0.90

    def test_cl_pre_trainer_model_load(self):
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f'Selected Device: {device}')

        # Hyperparameters
        batch_size = 2 if device == torch.device("cpu") else 4
        num_heads = 8
        hidden_dim = 768
        ff_dim = 2048
        num_layers = 6
        dropout_p = 0.1
        max_decoding_length = 16
        task_generator_indexes = [3]
        generator_range = 2 if device == torch.device("cpu") else 10
        number_of_batch = generator_range * len(task_generator_indexes)
        seed = 42
        add_bos_and_eos = True
        warmup = 400 # (number of batch * epoch) // 2
        max_rate= 0.00006
        lr_factor = 0.025

        # Initializing the data loader
        data_loader = DataLoader()

        data_loader_result = data_loader.create_data_loader_output(
            batch_size=batch_size,
            number_of_batch=number_of_batch,
            add_bos_and_eos=add_bos_and_eos,
            max_sequence_length=max_decoding_length,
            task_generator_indexes=task_generator_indexes,
            generator_indexes=[i for i in range(generator_range)],
            identifier=0,
            shuffle=True,
            seed=seed,
        )
        print(data_loader_result)

        corpus_io_parser_output = [item[Constants.IO_PARSER_OUTPUT] for item in data_loader_result]
        # Initialize category vocabulary builder instance
        category_vocab_builder = CategoryVocabBuilder(corpus_io_parser_output)
        category_vocab_size = len(category_vocab_builder.category_vocab_item_to_index.keys())

        print(f"Output token classification head count:"
              f" {len(category_vocab_builder.index_to_output_token_classification_head_vocab_item.keys())}\n"
              f"Output token classification head category type:"
              f" {category_vocab_builder.index_to_output_token_classification_head_vocab_item}")

        # Initialize output vocabulary builder instance
        output_vocab_builder = OutputVocabBuilder(
            corpus_of_io_parser_output=corpus_io_parser_output,
            index_to_output_token_classification_head_vocab_item=
            category_vocab_builder.index_to_output_token_classification_head_vocab_item
        )
        output_vocabularies = output_vocab_builder.index_to_output_vocabularies
        print(f"Output vocabularies count: {len(output_vocabularies.keys())}")

        # Creating the batch and masks
        batches, masks = BatchBuilder.construct_batches_for_cl_pre_trainer_with_data_loader(
            data_loader_result,
            batch_size=batch_size,
            max_decoder_sequence_length=max_decoding_length,
            is_generative_training=False,
            add_bos_and_eos=add_bos_and_eos,
        )
        # Initializing the CL pre trainer
        cl_pre_trainer = ClPreTrainer(
            batch_size=batch_size,
            hidden_dim=hidden_dim,
            ff_dim=ff_dim,
            num_heads=num_heads,
            num_layers=num_layers,
            max_decoding_length=max_decoding_length,
            dropout_p=dropout_p,
            category_vocab_size=category_vocab_size,
            index_to_output_vocabularies=output_vocabularies,
        )
        # Moved to CPU or GPU
        cl_pre_trainer.to(device)
        cl_pre_trainer.eval()

        # Initialize learning rate scheduler, optimizer and loss (note: the original paper uses label smoothing)
        optimizer = torch.optim.AdamW(
            cl_pre_trainer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01,
        )

        # Load the model...
        checkpoint_map = ClPreTrainerCheckPointManager.load_checkpoint_map(
            TestClPreTrainerTraining.BEST_PATH
        )
        # Load CL-Pre-Trainer states
        cl_pre_trainer.load_saved_model_from_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.CL_PRE_TRAINER_STATE,
            ),
        )
        # Load Optimizer states
        optimizer.load_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OPTIM_STATE,
            ),
        )
        start_epoch = ClPreTrainerCheckPointManager.get_checkpoint_item(
            checkpoint_map,
            ClPreTrainerCheckPointManager.EPOCH,
        )

        # Load Category map Output token classification heads state
        cl_pre_trainer.category_router.load_all_output_classification_head(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OUTPUT_TOKEN_CLASSIFICATION_HEADS_STATE,
            ),
        )

        print("Model loaded correctly...")

        scheduler = NoamOpt(
            cl_pre_trainer.hidden_dim,
            factor=lr_factor,
            warmup=warmup,
            optimizer=optimizer,
            max_rate=max_rate,
        )

        category_criterion = PreTrainerUtils.get_category_criterion(
            category_index_to_count=category_vocab_builder.index_to_count,
        )
        output_criterion_map = PreTrainerUtils.get_output_criterion_map(
            index_to_output_vocabularies=output_vocab_builder.index_to_output_vocabularies,
        )

        # Start training and verify ~zero loss and >90% accuracy on the last batch
        latest_batch_loss, latest_batch_accuracy, output_logits_map, execute_epoch = cl_pre_trainer_train(
            model=cl_pre_trainer,
            category_vocab_builder=category_vocab_builder,
            output_vocab_builder=output_vocab_builder,
            scheduler=scheduler,
            batches=batches,
            masks=masks,
            n_epochs=start_epoch,
            is_training=False,
            verbose_log=True,
            category_criterion=category_criterion,
            output_criterion_map=output_criterion_map,
            device=device
        )

        print(f"batch loss {latest_batch_loss.item()}")
        print(f"batch accuracy {latest_batch_accuracy}")
        self.assertEqual(latest_batch_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
        self.assertEqual(latest_batch_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)
        for index, output_logits_item in output_logits_map.items():
            output_loss = output_logits_item[CURRENT_BATCH_OUTPUT_LOSS]
            output_accuracy = output_logits_item[CURRENT_BATCH_OUTPUT_ACCURACY]
            self.assertEqual(output_loss.item() <= TestClPreTrainerTraining.accepted_loss_threshold, True)
            self.assertEqual(output_accuracy >= TestClPreTrainerTraining.accepted_accuracy_threshold, True)

test_suite = unittest.TestLoader().loadTestsFromTestCase(TestClPreTrainerTraining)
unittest.TextTestRunner(verbosity=2).run(test_suite)

Main Inference Code

In [3]:
from typing import List, Dict, Any
import copy

import torch
from torch import nn

from category_router.category_router import CategoryRouter
from cl_data.src.constants import Constants, SpecialTokens
from cl_pretrainer.batch_builder import BatchBuilder
from cl_pretrainer.cl_pre_trainer import ClPreTrainer
from cl_pretrainer.pre_trainer_checkpoint_manager import ClPreTrainerCheckPointManager
from cl_pretrainer.pre_trainer_utils import PreTrainerUtils
from data_loader.data_loader import DataLoader
from evaluation_metric.bleu import get_n_gram_weights, calculate_corpus_bleu_score
from evaluation_metric.perplexity import get_target_tokens_probability, calculate_batch_perplexity
from response_parser.response_parser import ResponseParser
from vocabulary_builder.category_vocabulary_builder import CategoryVocabBuilder
from vocabulary_builder.output_vocabulary_builder import OutputVocabBuilder
from response_parser.simple_response_parser import SimpleResponseParser


def cl_pre_trainer_inference_hub(
        model: nn.Module,
        category_vocab_builder: CategoryVocabBuilder,
        output_vocab_builder: OutputVocabBuilder,
        batches: Dict[str, List[List[List[dict]]]],
        masks: Dict[str, List[torch.Tensor]],
        max_decoding_length: int,
        device: torch.device = torch.device('cpu')
):
    target_batches = []
    predicted_batches = []
    output_logits_map_batches = []
    model.train(False)
    num_iters = 0
    for i, (src_batch, padding_mask, tgt_batch, future_mask, task_types, initial_token_count) in enumerate(
            zip(batches[BatchBuilder.ENCODER_IO_PARSER_OUTPUT_KEY],
                masks[BatchBuilder.PADDING_MASK_KEY],
                batches[BatchBuilder.DECODER_IO_PARSER_OUTPUT_KEY],
                masks[BatchBuilder.FUTURE_MASK_KEY],
                masks[BatchBuilder.TASK_TYPE_KEY],
                masks[BatchBuilder.INITIAL_TOKEN_COUNT_KEY],
                )
    ):
        # Initially we need at least 4 words for predicting the next word
        current_sequence_length = initial_token_count
        truncated_src_batch = [sequence_list[:current_sequence_length] for sequence_list in src_batch]
        truncated_src_batch_input = copy.deepcopy(truncated_src_batch)
        truncated_src_batch_input = [BatchBuilder.add_padding_and_eos(
            sequence_list,
            max_decoding_length,
            add_bos_and_eos=True,
            is_eos_finishing_token=False,
        ) for sequence_list in truncated_src_batch_input]
        # truncated_future_mask = BatchBuilder.construct_future_mask(current_sequence_length)

        output_logits_map = {}
        # range will be max_decoding_length - current_sequence_length
        # (We can add +1 but that will produce the garbage token)
        for index in range(max_decoding_length - current_sequence_length):
            try:
                # ~~~~~~~~~~~~~~~~~~~~~~~~~ Compute category probability ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                e_one = model.category_map_decoder.forward(
                    batch_io_parser_output=truncated_src_batch_input,
                    task_types=task_types,
                    # future_mask=truncated_future_mask,
                )

                category_probability, category_logits = model.category_map_classification_head.forward(e_one)
                predicted_category_map = category_vocab_builder.batch_decode(category_probability.tolist())
                # print(f"Predicted category probability values:"
                #       f" {predicted_category_map}")

                # ~~~~~~~~~~~~~~~~~~~~~~~~~~ Compute output token probability ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                e_two = model.output_token_decoder.forward(
                    batch_io_parser_output=truncated_src_batch_input,
                    task_types=task_types,
                    # future_mask=truncated_future_mask,
                )

                predicted_io_parser_output_without_token = PreTrainerUtils.convert_category_map_into_io_parser_output_without_token(
                    batch_category_map=predicted_category_map,
                )
                batch_route_ids = category_vocab_builder.batch_encoder_output_token_classification_head_vocab_items(
                    batch_io_parser_output=predicted_io_parser_output_without_token,
                )
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~  is_hub=True ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                # SO WE ARE ALSO USING output_vocab_builder.batch_decode_for_training
                output_logits_map = model.category_router.forward(
                    e_two=e_two,
                    batch_route_ids=batch_route_ids,
                    is_hub=True,
                )

                predicted_tokens_map = {}
                for output_classification_head_index, output_logits_item in output_logits_map.items():
                    current_head_output_probability = output_logits_item[CategoryRouter.OUTPUT_PROBABILITY]
                    # current_head_output_probability = current_head_output_probability[:, :-1]
                    current_head_predicted_output_token = output_vocab_builder.batch_decode_for_training(
                        output_classification_head_index,
                        current_head_output_probability.tolist(),
                    )
                    # print(f"Predicted token values for index: {output_classification_head_index} is \n"
                    #       f"{current_head_predicted_output_token}")

                    # Get the output token classification vocab item from index
                    current_output_token_classification_head_vocab_item = \
                        output_vocab_builder.index_to_output_vocabularies[output_classification_head_index][
                            OutputVocabBuilder.OUTPUT_TOKEN_CLASSIFICATION_HEAD_VOCAB_ITEM]

                    # Add item to predicted tokens map using the output token classification head vocab item as key
                    predicted_tokens_map[current_output_token_classification_head_vocab_item] = {
                        OutputVocabBuilder.PREDICTED_TOKEN_KEY: current_head_predicted_output_token,
                        OutputVocabBuilder.INDEX: output_classification_head_index,
                    }
                # print("\n")

                # Removed the teacher forcing and added the prediction to the src batch
                predicted_io_parser_output = PreTrainerUtils.recreate_io_parser_output_hub(predicted_category_map,
                                                                                           predicted_tokens_map,
                                                                                           start_from=1)

                truncated_src_batch = PreTrainerUtils.add_prediction_to_truncated_list(
                    predicted_io_parser_output,
                    truncated_src_batch,
                    current_sequence_length - 1 + index,
                )
                truncated_src_batch_input = copy.deepcopy(truncated_src_batch)
                truncated_src_batch_input = [BatchBuilder.add_padding_and_eos(
                    sequence_list,
                    max_decoding_length,
                    add_bos_and_eos=True,
                    is_eos_finishing_token=False,
                ) for sequence_list in truncated_src_batch_input]

                if truncated_src_batch[0][-1][Constants.TOKEN] == SpecialTokens.ENDING.value:
                    break
                # truncated_future_mask = BatchBuilder.construct_future_mask(current_sequence_length + index + 1)

            except Exception as e:
                print(f"An error occurred for batch: {i} word: {index} error: {e}")

        # Removing <BOS> from both tgt and predicted sentences
        tgt_batch = [sequence_list[1:len(truncated_src_batch[i])] for i, sequence_list in enumerate(tgt_batch)]
        truncated_src_batch = [sequence_list[1:] for sequence_list in truncated_src_batch]
        # print(f"Target batch: {tgt_batch}")
        # print(f"Predicted batch: {truncated_src_batch}\n")
        target_batches.append(tgt_batch)
        predicted_batches.append(truncated_src_batch)
        output_logits_map_batches.append(output_logits_map)
        num_iters += 1
        if num_iters == 100:
          break
        # break

    calculate_bleu_score(target_batches, predicted_batches)
    calculate_perplexity_score(target_batches, output_logits_map_batches, output_vocab_builder)
    print_response(predicted_batches)
    print("DONE")


def print_response(predicted_batches: List[List[List[dict]]]):
    for index, batch in enumerate(predicted_batches):
        parsed_response_list = ResponseParser.parse_corpus_io_parser_output(batch)
        print(f"For batch: {index}\n Parser response list is: {parsed_response_list} ")


def calculate_perplexity_score(
        target_batches: List[List[List[dict]]],
        output_logits_map_batches: List[dict[int, dict[str, Any]]],
        output_vocab_builder: OutputVocabBuilder,
):
    for batch_index, (target_batch, output_logits_map) in enumerate(zip(target_batches, output_logits_map_batches)):
        target_batch_extracted_token = PreTrainerUtils.extract_tokens(target_batch)
        batch_predicted_probabilities = get_target_tokens_probability(
            target_batch,
            output_logits_map,
            output_vocab_builder,
        )
        perplexity_score = calculate_batch_perplexity(
            target_batch_extracted_token,
            batch_predicted_probabilities
        )
        print(f"Perplexity Score of the {batch_index} th corpus is: {perplexity_score}")


def calculate_bleu_score(target_batches: List[List[List[dict]]], predicted_batches: List[List[List[dict]]]):
    for batch_index, (target_batch, predicted_batch) in enumerate(zip(target_batches, predicted_batches)):
        target_batch_extracted_token = PreTrainerUtils.extract_tokens(target_batch)
        predicted_batch_extracted_token = PreTrainerUtils.extract_tokens(predicted_batch)

        bleu_score = calculate_corpus_bleu_score(
            target_batch_extracted_token,
            predicted_batch_extracted_token,
            bleu_weights=get_n_gram_weights(2),
        )
        # Printing the raw response
        print("Target batch: ")
        deep_copied_tgt_list = copy.deepcopy(target_batch)
        SimpleResponseParser.print_response_to_console(deep_copied_tgt_list)
        print("\n")
        print("Predicted batch: ")
        deep_copied_predicted_list = copy.deepcopy(predicted_batch)
        SimpleResponseParser.print_response_to_console(deep_copied_predicted_list)
        print("\n")
        print(f"BLEU Score of the {batch_index} th corpus is: {bleu_score}")

Inference Test

In [4]:
# complete this first

import unittest


class TestClPreTrainerInference(unittest.TestCase):
    PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_best_v4.pth"
    # PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_best.pth"
    # PATH = "/content/drive/MyDrive/saved_models/cl_pre_trainer_generative_last.pth"

    accepted_loss_threshold = 0.09
    accepted_accuracy_threshold = 0.99

    def test_cl_pre_trainer_model_load_and_inference(self):
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f'Selected Device: {device}')

        # Hyperparameters
        # batch_size = 2 if device == torch.device("cpu") else 4
        # num_heads = 8
        # hidden_dim = 768
        # ff_dim = 2048
        # num_layers = 6
        # dropout_p = 0.1
        # max_decoding_length = 16
        # task_generator_indexes = [3]
        # generator_range = 2 if device == torch.device("cpu") else 10
        # number_of_batch = generator_range * len(task_generator_indexes)
        # seed = 42
        # add_bos_and_eos = True

        batch_size = 4 if device == torch.device("cpu") else 4
        num_heads = 8
        hidden_dim = 768
        ff_dim = 2048
        num_layers = 6
        dropout_p = 0.1
        max_decoding_length = 24
        task_generator_indexes = [0, 1, 2]
        generator_range = 20 if device == torch.device("cpu") else 20
        number_of_batch = generator_range * len(task_generator_indexes)
        seed = 42
        add_bos_and_eos = True
        training_batch_size = 1

        # Initializing the data loader
        data_loader = DataLoader()

        data_loader_result = data_loader.create_data_loader_output(
            batch_size=batch_size,
            number_of_batch=number_of_batch,
            add_bos_and_eos=add_bos_and_eos,
            max_sequence_length=max_decoding_length,
            task_generator_indexes=task_generator_indexes,
            generator_indexes=[i for i in range(generator_range)],
            identifier=0,
            shuffle=True,
            seed=seed,
        )
        # print(data_loader_result)
        batch_size = training_batch_size

        corpus_io_parser_output = [item[Constants.IO_PARSER_OUTPUT] for item in data_loader_result]

        # Initialize category vocabulary builder instance
        category_vocab_builder = CategoryVocabBuilder(corpus_io_parser_output)
        category_vocab_size = len(category_vocab_builder.category_vocab_item_to_index.keys())

        print(f"Output token classification head count:"
              f" {len(category_vocab_builder.index_to_output_token_classification_head_vocab_item.keys())}\n"
              f"Output token classification head category type:"
              f" {category_vocab_builder.index_to_output_token_classification_head_vocab_item}")

        # Initialize output vocabulary builder instance
        output_vocab_builder = OutputVocabBuilder(
            corpus_of_io_parser_output=corpus_io_parser_output,
            index_to_output_token_classification_head_vocab_item=
            category_vocab_builder.index_to_output_token_classification_head_vocab_item
        )
        output_vocabularies = output_vocab_builder.index_to_output_vocabularies
        print(f"Output vocabularies count: {len(output_vocabularies.keys())}")

        # Creating the batch and masks
        batches, masks = BatchBuilder.construct_batches_for_cl_pre_trainer_with_data_loader(
            data_loader_result,
            batch_size=batch_size,
            max_decoder_sequence_length=max_decoding_length,
            is_generative_training=False,
            add_bos_and_eos=add_bos_and_eos,
        )
        # Initializing the CL pre trainer
        cl_pre_trainer = ClPreTrainer(
            batch_size=batch_size,
            hidden_dim=hidden_dim,
            ff_dim=ff_dim,
            num_heads=num_heads,
            num_layers=num_layers,
            max_decoding_length=max_decoding_length,
            dropout_p=dropout_p,
            category_vocab_size=category_vocab_size,
            index_to_output_vocabularies=output_vocabularies,
        )
        cl_pre_trainer.eval()

        # Initialize learning rate scheduler, optimizer and loss (note: the original paper uses label smoothing)
        optimizer = torch.optim.Adam(
            cl_pre_trainer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9
        )

        # Load the model...
        checkpoint_map = ClPreTrainerCheckPointManager.load_checkpoint_map(
            TestClPreTrainerInference.PATH
        )
        # Load CL-Pre-Trainer states
        cl_pre_trainer.load_saved_model_from_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.CL_PRE_TRAINER_STATE,
            ),
        )
        # Load Optimizer states
        optimizer.load_state_dict(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OPTIM_STATE,
            ),
        )
        start_epoch = ClPreTrainerCheckPointManager.get_checkpoint_item(
            checkpoint_map,
            ClPreTrainerCheckPointManager.EPOCH,
        )

        # Load Category map Output token classification heads state
        cl_pre_trainer.category_router.load_all_output_classification_head(
            ClPreTrainerCheckPointManager.get_checkpoint_item(
                checkpoint_map,
                ClPreTrainerCheckPointManager.OUTPUT_TOKEN_CLASSIFICATION_HEADS_STATE,
            ),
        )

        print("Model loaded correctly...")

        # Call the inference method without teacher forcing
        cl_pre_trainer_inference_hub(
            model=cl_pre_trainer,
            category_vocab_builder=category_vocab_builder,
            output_vocab_builder=output_vocab_builder,
            batches=batches,
            masks=masks,
            max_decoding_length=max_decoding_length,
            device=device,
        )


test_suite = unittest.TestLoader().loadTestsFromTestCase(TestClPreTrainerInference)
unittest.TextTestRunner(verbosity=2).run(test_suite)


test_cl_pre_trainer_model_load_and_inference (__main__.TestClPreTrainerInference) ... 

Selected Device: cuda
Output token classification head count: 8
Output token classification head category type: {0: OutputTokenClassificationHeadVocabItem(category_type='special', category_subtype='word'), 1: OutputTokenClassificationHeadVocabItem(category_type='function', category_subtype='float'), 2: OutputTokenClassificationHeadVocabItem(category_type='float', category_subtype='default'), 3: OutputTokenClassificationHeadVocabItem(category_type='word', category_subtype='default'), 4: OutputTokenClassificationHeadVocabItem(category_type='integer', category_subtype='default'), 5: OutputTokenClassificationHeadVocabItem(category_type='function', category_subtype='integer'), 6: OutputTokenClassificationHeadVocabItem(category_type='function', category_subtype='default'), 7: OutputTokenClassificationHeadVocabItem(category_type='list', category_subtype='default')}
Output vocabularies count: 8


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
                "subType": "default",
                "subSubType": "none"
            },
            "position": 9
        },
        {
            "token": "integers",
            "category": {
                "type": "word",
                "subType": "default",
                "subSubType": "none"
            },
            "position": 10
        }
    ]
]


Predicted batch: 
[
    [
        {
            "token": "<function MathFunctions.factorial at 0x7f3975115e10>",
            "category": {
                "type": "function",
                "subType": "integer",
                "subSubType": "execute"
            },
            "position": 1
        },
        {
            "token": 8,
            "category": {
                "type": "integer",
                "subType": "default",
                "subSubType": "param_last"
            },
            "position": 2
        },
        {
            "token": "=",
            "ca

KeyboardInterrupt: 